In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain.chat_models import init_chat_model
from typing_extensions import TypedDict, Annotated
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langchain_core.tools import tool
from langgraph.prebuilt import ToolNode, tools_condition

In [3]:
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

In [4]:
class State(TypedDict):
    messages: Annotated[list, add_messages]

In [5]:
#in real world, this would be an API call to a stock price service
@tool
def get_stock_price(symbol: str) -> float:
    '''Return the current price of a stock given the symbol
    :param symbol: stock symbol
    :return: current price of the stock
    '''
    return {
        "MSFT": 300.0,
        "AAPL": 150.0,
        "AMZN": 3500.0,
        "RIL": 2000.0,
    }.get(symbol, 0.0)

tools = [get_stock_price]

llm = init_chat_model("llama-3.3-70b-versatile",model_provider="groq")
llm_with_tools = llm.bind_tools(tools)

In [6]:
def chatbot(state: State) -> State:
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

builder = StateGraph(State)

builder.add_node("chatbot_node", chatbot)
builder.add_node("tools", ToolNode(tools))

builder.add_edge(START, "chatbot_node")
builder.add_conditional_edges("chatbot_node", tools_condition)
builder.add_edge("tools", "chatbot_node")
# builder.add_edge(tools_condition, "tools")
builder.add_edge("chatbot_node", END)

graph = builder.compile(checkpointer=memory)

In [7]:
from IPython.display import Markdown
Markdown(f"```mermaid\n{graph.get_graph().draw_mermaid()}\n```")

```mermaid
---
config:
  flowchart:
    curve: linear
---
graph TD;
	__start__([<p>__start__</p>]):::first
	chatbot_node(chatbot_node)
	tools(tools)
	__end__([<p>__end__</p>]):::last
	__start__ --> chatbot_node;
	chatbot_node -.-> __end__;
	chatbot_node -.-> tools;
	tools --> chatbot_node;
	classDef default fill:#f2f0ff,line-height:1.2
	classDef first fill-opacity:0
	classDef last fill:#bfb6fc

```

In [ ]:
config = { 'configurable' : {'thread_id':'1'} }


# graph.invoke expects a list of dicts or list of strings as input, where each dict must have a 'content' key. The output is a dict with a 'messages' key, which is a list of dicts with 'content' keys. 
state = graph.invoke({"messages": ["I want to buy 20 AMZN stocks using current price. Then 15 MSFT. What will be the total cost?"]}, config=config)
print(state["messages"][-1].content)

The total cost will be 20 * $3500 + 15 * $300 = $70,000 + $4,500 = $74,500.


In [12]:
msg = "Using the current price tell me the total price of 10 RIL stocks and add it to previous total"

state = graph.invoke({"messages": [{"role":"user", "content": msg} ]}, config=config)

print(state["messages"][-1].content)

The total price of 10 RIL stocks is 10 * $2000 = $20,000. Adding this to the previous total: $74,500 + $20,000 = $94,500.
